### 関東地方の人口を予測する
重回帰分析を使い、関東地方の各県について、同じく「ある年の人口」から「その翌年の人口」を予測。

使用するモデルは線形回帰モデルだけでなく、「ランダムフォレスト回帰」と「サポートベクトル回帰」のモデルも使用。

重回帰分析は説明変数を複数持つ(例:東京都の人口、神奈川県の人口、埼玉県の人口、千葉県の人口…)

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
# ランダムフォレスト回帰
from sklearn.ensemble import RandomForestRegressor
# サポートベクトル回帰
from sklearn.svm import SVR

In [2]:
df = pd.read_csv("population_data/japan_population.csv")
df.head()

,集計年,都道府県コード,都道府県名,総人口
0,1960,1,北海道,5039206.0
1,1960,2,青森県,1426606.0
2,1960,3,岩手県,1448517.0
3,1960,4,宮城県,1743195.0
4,1960,5,秋田県,1335580.0


In [3]:
# 1960年から2015年までの関東地方各県の総人口数データを機械学習にかける。
# ある年の都道府県コードをカテゴリ値とした上で ダミー変数化したものと総人口数を説明変数X、その翌年の総人口数を目的変数yに設定する
# （Xは縦56×7行の横7列の二次元配列、yは縦56×7の長さの1次元配列としてndarrayを作成する）
X = np.zeros((56 * 7,7), dtype=np.uint32)
y = np.zeros(56 * 7, dtype=np.uint32)

cnt = 0
for i in range(56 * 47):
    pref_id = df.iloc[i, 1]
    population = df.iloc[i, 3]
    next_population = df.iloc[i + 47, 3]
    
    #pref_id 8(茨城県)から、pref_id 14(神奈川県)を計測データXに格納する。
    if pref_id >= 8 and pref_id <= 14:
        # pref_id 14(神奈川県)は、ダミー変数化の際に1としない。
        # すべてのカテゴリのダミー変数を用意すると、互いの変数に束縛関係が働き、回帰分析の精度低下の可能性がある（多重共線性）
        if pref_id < 14:
            # 計測データXに都県のダミーデータを格納する。値は1とする。8~13のpref_idを0~5として番号を振りなおす。  
            X[cnt][pref_id - 8] = 1
        
        # 計測データXの7番目の要素は、その年の人口とする。
        X[cnt][6] = population
        # 教師データyは、翌年の人口とする。
        y[cnt] = next_population  
        cnt += 1  

# Xを確認
X[0:10]

array([[      1,       0,       0,       0,       0,       0, 2047024],
       [      0,       1,       0,       0,       0,       0, 1513624],
       [      0,       0,       1,       0,       0,       0, 1578476],
       [      0,       0,       0,       1,       0,       0, 2430871],
       [      0,       0,       0,       0,       1,       0, 2306010],
       [      0,       0,       0,       0,       0,       1, 9683802],
       [      0,       0,       0,       0,       0,       0, 3443176],
       [      1,       0,       0,       0,       0,       0, 2053000],
       [      0,       1,       0,       0,       0,       0, 1513000],
       [      0,       0,       1,       0,       0,       0, 1581000]],
      dtype=uint32)

In [4]:
y[0:10]

array([2053000, 1513000, 1581000, 2497000, 2356000, 9967000, 3606000,
       2056000, 1513000, 1584000], dtype=uint32)

In [5]:
# 1960年から2009年までを訓練データ、
# 2010年以降のデータをテストデータとして分割する
# 50年 × 7県 = 350
X_train = X[:350]
X_test = X[350:]
y_train = y[:350]
y_test = y[350:]

In [6]:
# LinearRegressionで回帰モデルを作成
model1 = LinearRegression()
model1.fit(X_train, y_train)

LinearRegression()

In [7]:
# LinearRegressionで予測実行
y_pred1 = model1.predict(X_test).astype(np.uint32)
y_pred1

array([ 2976504,  2011342,  2011854,  7234625,  6247455, 13182582,
        9095368,  2967043,  2003902,  2005009,  7248612,  6248144,
       13219973,  9106668,  2954454,  1996155,  1998231,  7255390,
        6231681, 13254834,  9116352,  2944770,  1990345,  1990484,
        7267011,  6232650, 13325526,  9129909,  2935086,  1984534,
        1983705,  7285410,  6240397, 13414617,  9148308,  2925379,
        1978971,  1978006,  7304327,  6253631, 13527211,  9170788],
      dtype=uint32)

In [8]:
y_test

array([ 2960000,  2000000,  2001000,  7209000,  6217000, 13198000,
        9060000,  2947000,  1992000,  1994000,  7216000,  6200000,
       13234000,  9070000,  2937000,  1986000,  1986000,  7228000,
        6201000, 13307000,  9084000,  2927000,  1980000,  1979000,
        7247000,  6209000, 13399000,  9103000,  2916976,  1974255,
        1973115,  7266534,  6222666, 13515271,  9126214,  2905000,
        1966000,  1967000,  7289000,  6236000, 13624000,  9145000],
      dtype=uint32)

In [9]:
# RandomForestRegressorで回帰モデルを作成
model2 = RandomForestRegressor()
model2.fit(X_train, y_train)

RandomForestRegressor()

In [10]:
# RandomForestRegressorで予測実行
y_pred2 = model2.predict(X_test).astype(np.uint32)
y_pred2

array([ 2970303,  2011532,  2011919,  7259869,  6282497, 13102855,
        9029371,  2968312,  2004047,  2004773,  7259869,  6282497,
       13102855,  9029371,  2960853,  1996894,  2002222,  7259869,
        6266563, 13102855,  9029371,  2953093,  1991143,  1991073,
        7266928,  6266563, 13102855,  9029371,  2947477,  1985695,
        1984616,  7336408,  6277451, 13102855,  9029371,  2947474,
        1983512,  1983634,  7340878,  6298828, 13102855,  9029371],
      dtype=uint32)

In [11]:
# SVRで回帰モデルを作成
model3 = SVR()
model3.fit(X_train, y_train)

SVR()

In [12]:
# SVRで予測実行
y_pred3 = model3.predict(X_test).astype(np.uint32)
y_pred3

array([3928213, 3928193, 3928193, 3928350, 3928321, 3928364, 3928382,
       3928213, 3928192, 3928192, 3928350, 3928321, 3928364, 3928383,
       3928213, 3928192, 3928192, 3928350, 3928320, 3928363, 3928383,
       3928212, 3928192, 3928192, 3928350, 3928320, 3928362, 3928383,
       3928212, 3928192, 3928192, 3928351, 3928321, 3928361, 3928383,
       3928212, 3928192, 3928192, 3928351, 3928321, 3928359, 3928383],
      dtype=uint32)